<a href="https://colab.research.google.com/github/solcanalla/fiumark/blob/main/decisiontree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modelo: Decision Tree #



##Inicialización##

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.patches as mpatches
import matplotlib
import seaborn as sns
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import (
    LabelEncoder,
    OneHotEncoder,
)

In [ ]:
!git clone https://github.com/solcanalla/fiumark.git
%cd fiumark
!git pull origin main
import preprocessing as pp

Cloning into 'fiumark'...
remote: Enumerating objects: 92, done.
remote: Counting objects: 100% (92/92), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 92 (delta 46), reused 42 (delta 16), pack-reused 0
Unpacking objects: 100% (92/92), done.
/content/fiumark
From https://github.com/solcanalla/fiumark
 * branch            main       -> FETCH_HEAD
Already up to date.


##Pre procesamiento##

In [ ]:
pd.options.display.max_columns = None
df = pp.get_dataset()
df = pp.decisiontree_preprocessing(df)
X_train, X_test, y_train, y_test = pp.get_train_test_data(df)

##Entrenamiento##

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

params = {
    'max_depth': np.arange(1, 30),
    'min_samples_leaf': np.arange(1, 30),
    'min_samples_split': np.arange(1,30),
    'criterion': ["gini", "entropy"],
    'splitter': ["best","random"]
}

clf = DecisionTreeClassifier(random_state=42)

rgscv = RandomizedSearchCV(
    clf, params, n_iter=100, scoring='roc_auc', n_jobs=-1, cv=5, return_train_score=True
).fit(X_train, y_train)

In [ ]:
print(f"Best score: {rgscv.best_score_}")
print(f"Best params {rgscv.best_params_}")

Best score: 0.8529280105707441
Best params {'splitter': 'random', 'min_samples_split': 27, 'min_samples_leaf': 6, 'max_depth': 10, 'criterion': 'entropy'}


##Evaluación del modelo##

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, rgscv.predict(X_test)))

              precision    recall  f1-score   support

           0       0.78      0.91      0.84        70
           1       0.85      0.65      0.73        51

    accuracy                           0.80       121
   macro avg       0.81      0.78      0.79       121
weighted avg       0.81      0.80      0.80       121



##Predicción##

In [ ]:
X_holdout = pd.read_csv('https://drive.google.com/uc?export=download&id=1I980-_K9iOucJO26SG5_M8RELOQ5VB6A')
X_holdout = pp.decisiontree_preprocessing(X_holdout)
df2 = pd.DataFrame(rgscv.predict(X_holdout),columns=['volveria'])
df3 = pd.concat([X_holdout.id_usuario,df2],axis=1)
df3.to_csv('decisiontree.csv')
df3

,id_usuario,volveria
0,650,0
1,234,0
2,31,1
3,552,1
4,411,0
...,...,...
85,354,0
86,82,0
87,172,1
88,8,1
